In [1]:
import geojson
import json
import requests
import pandas as pd
import time
import os

In [8]:
# Load the CSV file
csv_file_path = 'land_plots.csv'
df = pd.read_csv(csv_file_path)

In [ ]:
# URL for the API
url = 'https://plot.search.api.ongeo.pl/1.0/autocomplete'

save_path = '/Users/anastasiiamorozova/Downloads/działki'
processed_files = set(os.listdir(save_path))


for index, row in df.iterrows():
    identifier = row['Identyfikator działki']
    nr_działki = row['Nr działki']
    json_filename = f"{nr_działki}.json"
    file_path = os.path.join(save_path, json_filename)
    
        # Skip processing if the file already exists
    if json_filename in processed_files:
        print(f"Skipping {nr_działki}, already processed.")
        continue

    params = {
            'query': identifier,
            'api_key': 'a9968cb3-5b6e-4c6a-a06c-97904fd58f4b',
            'maxresults': '5',
            'additionalData': 'boundary'
        }
    headers = {
            'Referer': 'https://geoportal-krajowy.pl/'
        }

        # Make the GET request
    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        with open(file_path, 'w') as file:
            json.dump(response.json(), file, indent=4)
        print(f"Data for {nr_działki} saved successfully.")
    else:
        print(f"Failed for {nr_działki}) with status code {response.status_code}")

    # Sleep for 5 seconds after each request
    time.sleep(5)

In [16]:
# Define the directory containing the JSON files
json_directory_path = '/Users/anastasiiamorozova/Downloads/działki'  

# Initialize an empty GeoJSON FeatureCollection
geojson = {
    "type": "FeatureCollection",
    "features": []
}

# Loop through every file in the directory
for filename in os.listdir(json_directory_path):
    if filename.endswith('.json'):  # Check if the file is a JSON file
        file_path = os.path.join(json_directory_path, filename)
        
        # Open and read the JSON file
        with open(file_path, 'r') as file:
            data = json.load(file)
        
        # Process each item in the JSON array (assuming each file contains an array)
        for item in data:
            if "boundary" in item:
                # Create a GeoJSON feature using the boundary
                feature = {
                    "type": "Feature",
                    "properties": {
                        "label": item.get("label", "No Label"),
                        "area": item.get("area", None),
                        "globalId": item.get("globalId", "No Global ID")
                        # add more properties if needed
                    },
                    "geometry": item["boundary"]
                }
                geojson["features"].append(feature)

# Save the consolidated GeoJSON 
geojson_file_path = '/Users/anastasiiamorozova/Downloads/land_plots.geojson'
with open(geojson_file_path, 'w') as f:
    json.dump(geojson, f, indent=4)

print("Combined GeoJSON file has been created and saved.")


Combined GeoJSON file has been created and saved.
